In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature_eng.scalers import ranged_scaler
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [3]:
from tkgngc.embeddings import PretrainedTKGEmbeddingWithTimestamps
from tkgngc.data_processing import TKGNGCDataProcessor
from tkgngc.model import train_model

In [4]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [5]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [6]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.571429,0.25,0.5,0.5,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.40973,0.246024,0.14197,0.112819,0.550194,0.406691,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.40671,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726,0.0,0.0,4


In [7]:
cats_df = cats_df.to_pandas()

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [10]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [11]:
train_df.head()

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")


In [13]:
device

device(type='cuda')

In [14]:
train_df.columns

Index(['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1'],
      dtype='object')

In [15]:
tkgnc_data = TKGNGCDataProcessor(train_df,device,num_timestamps=20, lags=1)

In [16]:
pretrained_tkg = PretrainedTKGEmbeddingWithTimestamps(
    num_entities=int(tkgnc_data.entity_indices.max().item()+1),
    num_relations=int(tkgnc_data.relation_indices.max().item()+1),
    embedding_dim=17,
    num_timestamps=tkgnc_data.num_timestamps,
).to(device)

In [17]:
quads = (
    tkgnc_data.entity_indices[:-1],  # Head entities
    tkgnc_data.relation_indices,  # Relations
    tkgnc_data.entity_indices[1:],  # Tail entities (shifted example)
    tkgnc_data.timestamp_indices[:-1],  # Timestamps
)


In [18]:
#pretrained_tkg.pretrain(quads, learning_rate=0.01, epochs=1000)
#torch.save(pretrained_tkg.state_dict(), 'pretrained_tkg')
pretrained_tkg.load_state_dict(torch.load('pretrained_tkg'))

<All keys matched successfully>

In [19]:
from CARAT.data_processing import *
from CARAT.model import *
from CARAT.causal_inference import causal_inference

Using device: cuda


C:\Users\jlowh\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [21]:
cols = list(train_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

num_nodes = len(train_df.columns)

new_metadata = []
A0 = get_adjacency(cols,causal_indices,non_causal_indices,num_nodes)    
BATCH_SIZE = 64

model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
from utils.utils import set_seed
set_seed()
n_correct = 0
method2_correct=0
total_checked = 0
for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    print('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]
    print('Anomaly: ' + anomaly)
    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    mod_df = test_df[(test_df.index>= anomaly_time) & (test_df.index<= end_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]
    num_nodes = len(mod_df.columns)

    start_len = mod_df.shape[0]
    if start_len >1000:
        start_len = 1000

    start_time = anomaly_time- timedelta(seconds=start_len)

    normal_df = test_df[(test_df.index>= start_time) & (test_df.index< anomaly_time)]

    sample_data = TKGNGCDataProcessor(mod_df,device,num_timestamps=20, lags=1)
    model_data = create_granger_gat_data(pretrained_tkg,sample_data)
    model_data.retrain_tkg()
    features = model_data.time_series_data
    model_name = 'model_category_'+str(row['category'])
    """model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))"""
    model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    data = create_lagged_features(features, 3, pad_value=0)
    ee = create_lagged_features(model_data.entity_emb, 3, pad_value=0)
    tt = create_lagged_features(model_data.timestamp_emb, 3, pad_value=0)
        
    dataset = TimeSeriesDataset(data, ee, tt)
    dataloader = DataLoader(dataset,batch_size = BATCH_SIZE, shuffle=False)
    loss = train_causal_vae(model, optimizer, dataloader, A0, num_epochs=1000)

    model.eval()
    with torch.no_grad():
        _, _, _, learned_adj_matrix = model(data,ee,tt, num_nodes=num_nodes)
        causal_graph = learned_adj_matrix.cpu().numpy()


    adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
    candidates = adj_data[anomaly].sort_values(ascending=False)
    candidates = candidates[candidates.index.isin( potential_causes)]
    #A0 = learned_adj_matrix
    #try:
        
    ci = causal_inference(causal_graph, mod_df, normal_df,potential_causes)
    ci.infer_causal_path(anomaly)  

    if len(ci.causal_factors) >= 3:
        potential_cause1 = ci.causal_factors[0][0]
        potential_cause2 = ci.causal_factors[1][0]
        potential_cause3 = ci.causal_factors[2][0]

    elif len(ci.causal_factors) == 2:
        potential_cause1 = ci.causal_factors[0][0]
        potential_cause2 = ci.causal_factors[1][0]
        potential_cause3 = 'NA'
    elif len(ci.causal_factors) == 1:
        potential_cause1 = ci.causal_factors[0][0]
        potential_cause2 = "NA"
        potential_cause3 = 'NA'
    else:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]

    if len(ci.causal_effects) >= 3:
        causal_effet1 = ci.causal_effects[0][0]
        causal_effet2 = ci.causal_effects[1][0]
        causal_effet3 = ci.causal_effects[2][0]
    elif len(ci.causal_effects) == 2:
        causal_effet1 = ci.causal_effects[0][0]
        causal_effet2 = ci.causal_effects[1][0]
        causal_effet3 = 'NA'
    elif len(ci.causal_effects) == 1:
        causal_effet1 = ci.causal_effects[0][0]
        causal_effet2 = 'NA'
        causal_effet3 = 'NA'
    else:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]

    """except:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]
        causal_effet1 = candidates.index[0]
        causal_effet2 = candidates.index[1]
        causal_effet3 = candidates.index[2]"""

   
    if root_cause in [potential_cause1,potential_cause2,potential_cause3]:
        n_correct+=1

    if root_cause in [causal_effet1,causal_effet2,causal_effet3]:
        method2_correct+=1

    if root_cause == potential_cause1:
        row['cause_1'] = 1
    if root_cause == potential_cause2:
        row['cause_2'] = 1
    if root_cause == potential_cause3:
        row['cause_3'] = 1
    new_metadata.append(row)
    if root_cause in [potential_cause1 , potential_cause2 , potential_cause3]:
        print('ROOT CAUSE FOUND!')
    print(root_cause + '-->' + potential_cause1 + ' | ' + potential_cause2 + ' | ' + potential_cause3)
    print(root_cause + '-->' + causal_effet1 + ' | ' + causal_effet2 + ' | ' + causal_effet3)

    print("Model Accuracy Method 1: " + str(round( (n_correct/total_checked)*100,2)) + '%' )
    print("Model Accuracy Method 2: " + str(round( (method2_correct/total_checked)*100,2)) + '%' )
    
    print('--------------------------------------------------------------------------------------------------------------------------------')




Model: 0
Anomaly: cfo1
Epoch 0, Loss: 0.0011
Epoch 0: Average Loss = 150.2616, Best Loss = 150.2616
Epoch 100: Average Loss = 31.1117, Best Loss = 30.8912
Early stopping triggered after 139 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 213/213 [00:13<00:00, 16.21it/s]


ROOT CAUSE FOUND!
bso3-->bso3 | bfo1 | bso1
bso3-->bfo1 | bed1 | bso2
Model Accuracy Method 1: 100.0%
Model Accuracy Method 2: 0.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 1
Anomaly: cfo1
Epoch 0, Loss: 0.0015
Epoch 0: Average Loss = 61.3180, Best Loss = 61.3180
Early stopping triggered after 41 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 263/263 [00:30<00:00,  8.54it/s]


bso3-->bfo1 | bso1 | bed1
bso3-->bfo1 | bfo2 | bso3
Model Accuracy Method 1: 50.0%
Model Accuracy Method 2: 50.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 2
Anomaly: cso1
Epoch 0, Loss: 0.0005
Epoch 0: Average Loss = 151.0029, Best Loss = 151.0029
Epoch 100: Average Loss = 29.4992, Best Loss = 29.4992
Early stopping triggered after 128 epochs. Restoring best model.


...

bso3-->bfo1 | bso1 | bed1
bso3-->bfo1 | bed1 | bso1
Model Accuracy Method 1: 39.84%
Model Accuracy Method 2: 44.72%
--------------------------------------------------------------------------------------------------------------------------------
Model: 123
Anomaly: cso1
Epoch 0, Loss: 0.0000
Epoch 0: Average Loss = 143.3532, Best Loss = 143.3532
Epoch 100: Average Loss = 31.0627, Best Loss = 31.0199
Early stopping triggered after 107 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 348/348 [00:45<00:00,  7.66it/s]


ROOT CAUSE FOUND!
bfo2-->bfo2 | bfo1 | bso3
bfo2-->bfo2 | bso1 | bso2
Model Accuracy Method 1: 40.32%
Model Accuracy Method 2: 45.16%
--------------------------------------------------------------------------------------------------------------------------------
Model: 124
Anomaly: cfo1
Epoch 0, Loss: 0.0001
Epoch 0: Average Loss = 129.6473, Best Loss = 129.6473
Epoch 100: Average Loss = 32.1625, Best Loss = 32.0133
Early stopping triggered after 152 epochs. Restoring best model.


Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:02<00:00,  6.88it/s]


KeyboardInterrupt: 